# Data Loading
In this section we will import the necessary library and loaded the dataset that is going to be used in this project.

In [1]:
import pandas as pd
import numpy as np

In [2]:
df = pd.read_csv(r"D:\Downloads\books.csv\books.csv")
df

,isbn13,isbn10,title,subtitle,authors,categories,thumbnail,description,published_year,average_rating,num_pages,ratings_count
0,9780002005883,0002005883,Gilead,NaN,Marilynne Robinson,Fiction,http://books.google.com/books/content?id=KQZCP...,A NOVEL THAT READERS and critics have been eag...,2004.0,3.85,247.0,361.0
1,9780002261982,0002261987,Spider's Web,A Novel,Charles Osborne;Agatha Christie,Detective and mystery stories,http://books.google.com/books/content?id=gA5GP...,A new 'Christie for Christmas' -- a full-lengt...,2000.0,3.83,241.0,5164.0
2,9780006163831,0006163831,The One Tree,NaN,Stephen R. Donaldson,American fiction,http://books.google.com/books/content?id=OmQaw...,Volume Two of Stephen Donaldson's acclaimed se...,1982.0,3.97,479.0,172.0
3,9780006178736,0006178731,Rage of angels,NaN,Sidney Sheldon,Fiction,http://books.google.com/books/content?id=FKo2T...,"A memorable, mesmerizing heroine Jennifer -- b...",1993.0,3.93,512.0,29532.0
4,9780006280897,0006280897,The Four Loves,NaN,Clive Staples Lewis,Christian life,http://books.google.com/books/content?id=XhQ5X...,Lewis' work on the nature of love divides love...,2002.0,4.15,170.0,33684.0
...,...,...,...,...,...,...,...,...,...,...,...,...
6805,9788185300535,8185300534,I Am that,Talks with Sri Nisargadatta Maharaj,Sri Nisargadatta Maharaj;Sudhakar S. Dikshit,Philosophy,http://books.google.com/books/content?id=Fv_JP...,This collection of the timeless teachings of o...,1999.0,4.51,531.0,104.0
6806,9788185944609,8185944601,Secrets Of The Heart,NaN,Khalil Gibran,Mysticism,http://books.google.com/books/content?id=XcrVp...,NaN,1993.0,4.08,74.0,324.0
6807,9788445074879,8445074873,Fahrenheit 451,NaN,Ray Bradbury,Book burning,NaN,NaN,2004.0,3.98,186.0,5733.0
6808,9789027712059,9027712050,The Berlin Phenomenology,NaN,Georg Wilhelm Friedrich Hegel,History,http://books.google.com/books/content?id=Vy7Sk...,Since the three volume edition ofHegel's Philo...,1981.0,0.00,210.0,0.0


# Creating the Text Representation
In this section i created a function that combines the book's metadata (title, authors, description, etc.) into a structured text format for each book. This unified format will make the book information easier to process and analyze.

In [3]:
def text_rep(row):
    text_rep = f"""Title:{row['title']}
Authors:{row['authors']}
Description:{row['description']}
Categories:{row['categories']}
Publishing Year:{row['published_year']}
Average Rating{row['average_rating']}
Number of Pages: {row['num_pages']}
"""
    return text_rep

In [4]:
print(df.iloc[:5].apply(text_rep,axis=1).values[0])

Title:Gilead
Authors:Marilynne Robinson
Description:A NOVEL THAT READERS and critics have been eagerly anticipating for over a decade, Gilead is an astonishingly imagined story of remarkable lives. John Ames is a preacher, the son of a preacher and the grandson (both maternal and paternal) of preachers. It’s 1956 in Gilead, Iowa, towards the end of the Reverend Ames’s life, and he is absorbed in recording his family’s story, a legacy for the young son he will never see grow up. Haunted by his grandfather’s presence, John tells of the rift between his grandfather and his father: the elder, an angry visionary who fought for the abolitionist cause, and his son, an ardent pacifist. He is troubled, too, by his prodigal namesake, Jack (John Ames) Boughton, his best friend’s lost son who returns to Gilead searching for forgiveness and redemption. Told in John Ames’s joyous, rambling voice that finds beauty, humour and truth in the smallest of life’s details, Gilead is a song of celebration an

In [5]:
df['text_rep'] = df.apply(text_rep,axis=1)

# Generating the Embeddings with Ollama
In this section each book's text_rep is sent to a locally running Ollama server using the nomic-embed-text model.
We also create a FAISS index and add all the book embeddings, the index will then be saved for future use and reloaded as needed.

In [14]:
import faiss
import requests

dim = 768 
index = faiss.IndexFlatL2(dim)
X = np.zeros((len(df['text_rep']), dim), dtype='float32')

#### This is the part where each text representation is sent to Ollama to generate vector embeddings

In [16]:
for i, representation in enumerate(df['text_rep']):
    if i % 100 == 0:
        print(i)
        
    res = requests.post('http://localhost:11434/api/embeddings',
                        json = {
                            'model':'nomic-embed-text',
                            'prompt': representation
                        })
    
    embedding = res.json()['embedding']

    X[i] = np.array(embedding)

index.add(X)

0
100
200
300
400
500
600
700
800
900
1000
1100
1200
1300
1400
1500
1600
1700
1800
1900
2000
2100
2200
2300
2400
2500
2600
2700
2800
2900
3000
3100
3200
3300
3400
3500
3600
3700
3800
3900
4000
4100
4200
4300
4400
4500
4600
4700
4800
4900
5000
5100
5200
5300
5400
5500
5600
5700
5800
5900
6000
6100
6200
6300
6400
6500
6600
6700
6800


In [18]:
faiss.write_index(index, 'index2')

In [21]:
index = faiss.read_index('index')

# Simulating The System
In this section, we will be simulating how the recommendation system is performed to test the system's ability to generate meaningful book recommendations based on the user's selection.
The vector will be passed to FAISS, which returns the top 5 most similar books. These recommendations are based entirely on the semantic content of the book, not just keywords.

In [23]:
df[df.title.str.contains('Friends')]

,isbn13,isbn10,title,subtitle,authors,categories,thumbnail,description,published_year,average_rating,num_pages,ratings_count,text_rep
502,9780064420808,0064420809,Little House Friends,NaN,Heather Henson;Laura Ingalls Wilder,Juvenile Fiction,http://books.google.com/books/content?id=gMZbA...,Laura Ingalls shares adventures and good times...,1998.0,3.99,80.0,70.0,Title:Little House Friends\nAuthors:Heather He...
612,9780099498599,0099498596,Circle of Friends,NaN,Maeve Binchy,College attendance,http://books.google.com/books/content?id=P4-i_...,"It began with Benny Hogan and Eve Malone, grow...",2006.0,4.02,722.0,50668.0,Title:Circle of Friends\nAuthors:Maeve Binchy\...
986,9780142300848,0142300845,"Oliver and Albert, Friends Forever",NaN,Jean Van Leeuwen,Juvenile Fiction,http://books.google.com/books/content?id=-EBFs...,"Oliver makes friends with Albert, the new boy ...",2002.0,3.62,48.0,22.0,"Title:Oliver and Albert, Friends Forever\nAuth..."
1776,9780345323903,0345323904,With Friends Like These,NaN,Alan Dean Foster,Fiction,http://books.google.com/books/content?id=IKUcA...,Willie Whitehorse could have been just another...,1984.0,3.94,236.0,2233.0,Title:With Friends Like These\nAuthors:Alan De...
2490,9780394895833,0394895835,Baby's Animal Friends,NaN,Phoebe Dunn,Juvenile Fiction,http://books.google.com/books/content?id=ytIAZ...,Photographs capture the special relationship b...,1988.0,3.40,28.0,15.0,Title:Baby's Animal Friends\nAuthors:Phoebe Du...
4373,9780743272773,0743272773,How to Win Friends and Influence People for Te...,NaN,Donna Dale Carnegie,Self-Help,http://books.google.com/books/content?id=eIc2l...,"Donna Dale Carnegie, daughter of the late moti...",2005.0,3.90,208.0,426.0,Title:How to Win Friends and Influence People ...
4533,9780749307844,0749307846,How to Win Friends and Influence People,NaN,Dale Carnegie,Conduct of life,http://books.google.com/books/content?id=aO7CQ...,Dale Carnegie aims to show how to makes friend...,1990.0,4.18,256.0,199.0,Title:How to Win Friends and Influence People\...
5115,9780810958623,0810958627,Ruby Gloom's Guide to Friendship,NaN,Mighty Fine Inc.,Juvenile Nonfiction,http://books.google.com/books/content?id=FYrcu...,"If Ruby Gloom's friends seem somewhat unusual,...",2005.0,4.34,72.0,37.0,Title:Ruby Gloom's Guide to Friendship\nAuthor...


In [25]:
fave_book = df.iloc[4500]

In [27]:
print(fave_book['text_rep'])

Title:Solo
Authors:Jill Mansell
Description:Love is always just around the corner in a Jill Mansell novel - with a few surprises and a lot of humour on the way to happiness When Tessa Duvall, a struggling artist, reluctantly agreed to accompany her best friend Holly to a party at the elegant Charrington Grange Hotel, she had every intention of sneaking off early. For parties full of strangers bored the knickers off Tessa and this one proved to be no exception - until she encountered Ross Monahan, whose wicked reputation was as high profile as the hotel he owned and ran with such panache. But whilst Holly set about ensnaring his reluctant brother Max, Tessa simply accepted Ross for what he was, a sensational one-night stand...until she realised, weeks later, that one-night stands can have far-reaching consequences...
Categories:Chick lit
Publishing Year:2002.0
Average Rating3.66
Number of Pages: 421.0



In [31]:
res = requests.post('http://localhost:11434/api/embeddings',
                        json = {
                            'model':'nomic-embed-text',
                            'prompt': representation
                        })

In [34]:
embedding = np.array([res.json()['embedding']], dtype='float32')

In [36]:
D,I = index.search(embedding, 5)

In [37]:
best_matches = np.array(df['text_rep'])[I.flatten()]

In [42]:
for i in best_matches:
    print(i)
    print()

Title:'I'm Telling You Stories'
Authors:Helena Grice;Tim Woods
Description:This is a jubilant and rewarding collection of Winterson scholarship--a superb group of essays from a host of fine authors.
Categories:Literary Criticism
Publishing Year:1998.0
Average Rating3.7
Number of Pages: 136.0


Title:The Polysyllabic Spree
Authors:Nick Hornby
Description:Shares selections from the author's column in "The Believer" that comment on literature he has found interesting, and features passages from the material discussed.
Categories:Literary Criticism
Publishing Year:2004.0
Average Rating3.73
Number of Pages: 143.0


Title:The World and Other Places
Authors:Jeanette Winterson
Description:Presents a collection of seventeen stories spanning the author's career.
Categories:Fiction
Publishing Year:2000.0
Average Rating3.87
Number of Pages: 240.0


Title:McSweeney's Enchanted Chamber of Astonishing Stories
Authors:Michael Chabon;Michael Mignola
Description:Presents an anthology of short fiction fr